using ann to predict all stock prices--

In [83]:
import sys



In [84]:
import nselib as nse
from  nselib import capital_market 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.src.models.sequential import Sequential
from keras.src.layers.core.dense import Dense
from datetime import datetime, timedelta
import yfinance as yf
import time
import requests



In [85]:
start_date = '2005-07-23'
end_date = '2022-07-23'
ticker='SBIN.NS'
# Get the data
stock = yf.download(ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [86]:
stock.to_csv(f"{ticker}_data.csv", index=False)

In [87]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4193 non-null   float64
 1   High       4193 non-null   float64
 2   Low        4193 non-null   float64
 3   Close      4193 non-null   float64
 4   Adj Close  4193 non-null   float64
 5   Volume     4193 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 229.3 KB


In [88]:
data=stock

caculating sma , ema , wma (10 day)

In [89]:
# sma

print(data.columns)
print(data['Close'])


# Calculate the 10-day SMA
data['10_SMA'] = data['Close'].rolling(window=10, min_periods=10).mean()

# Display the last few rows to check the calculations
print(data[[ 'Close', '10_SMA']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
Date
2005-07-25     70.122467
2005-07-26     70.919746
2005-07-27     73.137016
2005-07-28     73.137016
2005-07-29     75.505257
                 ...    
2022-07-18    490.299988
2022-07-19    497.950012
2022-07-20    508.600006
2022-07-21    512.049988
2022-07-22    513.700012
Name: Close, Length: 4193, dtype: float64
                 Close      10_SMA
Date                              
2022-07-18  490.299988  483.580002
2022-07-19  497.950012  486.160004
2022-07-20  508.600006  489.025003
2022-07-21  512.049988  491.555002
2022-07-22  513.700012  494.070004
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4193 non-null   float64
 1   High       4193 non-null   float64
 2   Low        4193 non-null   float64
 3   Close      4193 non-null 

In [90]:
# wma-
weights = np.arange(1, 11)

# Function to calculate WMA for each rolling window
def calc_wma(values):
    return np.dot(values, weights) / weights.sum()

# Calculate the 10-day WMA using the custom function
data['10_WMA'] = data['Close'].rolling(window=10, min_periods=10).apply(calc_wma, raw=True)

# Display the last few rows to check the SMA and WMA calculations
print(data[['Close', '10_SMA', '10_WMA']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())

                 Close      10_SMA      10_WMA
Date                                          
2022-07-18  490.299988  483.580002  484.589092
2022-07-19  497.950012  486.160004  487.201822
2022-07-20  508.600006  489.025003  491.281822
2022-07-21  512.049988  491.555002  495.468183
2022-07-22  513.700012  494.070004  499.494548
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4193 non-null   float64
 1   High       4193 non-null   float64
 2   Low        4193 non-null   float64
 3   Close      4193 non-null   float64
 4   Adj Close  4193 non-null   float64
 5   Volume     4193 non-null   int64  
 6   10_SMA     4184 non-null   float64
 7   10_WMA     4184 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 294.8 KB
None


In [91]:
#ema-

# Define the window size for EMA
window = 10

# Calculate the smoothing factor (alpha)
alpha = 2 / (window + 1)

# Function to calculate EMA
def calculate_ema(series, alpha):
    ema = series.ewm(span=window, adjust=False).mean()
    return ema

# Calculate the 10-day EMA
data['10_EMA'] = calculate_ema(data['Close'], alpha)

# Display the last few rows to check the calculations
print(data[[ 'Close', '10_SMA', '10_WMA', '10_EMA']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())

                 Close      10_SMA      10_WMA      10_EMA
Date                                                      
2022-07-18  490.299988  483.580002  484.589092  481.322522
2022-07-19  497.950012  486.160004  487.201822  484.345702
2022-07-20  508.600006  489.025003  491.281822  488.755576
2022-07-21  512.049988  491.555002  495.468183  492.990923
2022-07-22  513.700012  494.070004  499.494548  496.756212
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4193 non-null   float64
 1   High       4193 non-null   float64
 2   Low        4193 non-null   float64
 3   Close      4193 non-null   float64
 4   Adj Close  4193 non-null   float64
 5   Volume     4193 non-null   int64  
 6   10_SMA     4184 non-null   float64
 7   10_WMA     4184 non-null   float64
 8   10_EMA     4193 non-null   float64
dtypes: float64(8), int

In [92]:
window = 10

# Function to calculate the Stochastic %K with EMA using only Close
def calculate_stochastic_k(data, window):
    # Calculate the rolling high and low from the Close over the specified window
    high_roll = data['Close'].rolling(window=window, min_periods=1).max()
    low_roll = data['Close'].rolling(window=window, min_periods=1).min()

    # Calculate the %K using the typical formula
    stochastic_k = (data['Close'] - low_roll) / (high_roll - low_roll) * 100
    
    # Apply EMA to the %K values
    stochastic_k_ema = stochastic_k.ewm(span=window, adjust=False).mean()
    
    return stochastic_k_ema

# Assuming your DataFrame is named 'data' and contains 'Close' column
data['Stochastic_K'] = calculate_stochastic_k(data, window)

# Display the last few rows to check the calculations
print(data[[ 'Close', 'Stochastic_K']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())

                 Close  Stochastic_K
Date                                
2022-07-18  490.299988     81.651989
2022-07-19  497.950012     84.987991
2022-07-20  508.600006     87.717447
2022-07-21  512.049988     89.950639
2022-07-22  513.700012     91.777795
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          4193 non-null   float64
 1   High          4193 non-null   float64
 2   Low           4193 non-null   float64
 3   Close         4193 non-null   float64
 4   Adj Close     4193 non-null   float64
 5   Volume        4193 non-null   int64  
 6   10_SMA        4184 non-null   float64
 7   10_WMA        4184 non-null   float64
 8   10_EMA        4193 non-null   float64
 9   Stochastic_K  4192 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 360.3 KB
None


In [93]:
# stochastic D-
window = 10
stochastic_d_window = 3  # Commonly, %D is a 3-period moving average of %K

# Function to calculate Stochastic %D (EMA of %K)
def calculate_stochastic_d(stochastic_k_ema, window):
    # Apply EMA to the %K EMA values to get %D
    stochastic_d = stochastic_k_ema.ewm(span=window, adjust=False).mean()
    return stochastic_d

# Assuming your DataFrame is named 'data' and contains 'Close' column
data['Stochastic_K'] = calculate_stochastic_k(data, window)
data['Stochastic_D'] = calculate_stochastic_d(data['Stochastic_K'], stochastic_d_window)

# Display the last few rows to check the calculations
print(data[[  'Close', 'Stochastic_K', 'Stochastic_D']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())


                 Close  Stochastic_K  Stochastic_D
Date                                              
2022-07-18  490.299988     81.651989     82.317595
2022-07-19  497.950012     84.987991     83.652793
2022-07-20  508.600006     87.717447     85.685120
2022-07-21  512.049988     89.950639     87.817879
2022-07-22  513.700012     91.777795     89.797837
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          4193 non-null   float64
 1   High          4193 non-null   float64
 2   Low           4193 non-null   float64
 3   Close         4193 non-null   float64
 4   Adj Close     4193 non-null   float64
 5   Volume        4193 non-null   int64  
 6   10_SMA        4184 non-null   float64
 7   10_WMA        4184 non-null   float64
 8   10_EMA        4193 non-null   float64
 9   Stochastic_K  4192 non-null   float64
 1

In [94]:
# rsi

# Define the window size for RSI calculation
rsi_window = 14  # Typically, a 14-period window is used

# Function to calculate RSI using EMA
def calculate_rsi(data, window):
    # Calculate the price changes
    delta = data['Close'].diff()

    # Separate gains and losses
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)

    # Calculate the exponential moving averages (EMA) of gains and losses
    avg_gain = gains.ewm(span=window, adjust=False).mean()
    avg_loss = losses.ewm(span=window, adjust=False).mean()

    # Calculate the Relative Strength (RS)
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Assuming your DataFrame is named 'data' and contains 'Close' column
data['RSI'] = calculate_rsi(data, rsi_window)

# Display the last few rows to check the calculations
print(data[[  'Close', 'RSI']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())

                 Close        RSI
Date                             
2022-07-18  490.299988  69.445675
2022-07-19  497.950012  75.577694
2022-07-20  508.600006  81.531522
2022-07-21  512.049988  83.073871
2022-07-22  513.700012  83.819569
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          4193 non-null   float64
 1   High          4193 non-null   float64
 2   Low           4193 non-null   float64
 3   Close         4193 non-null   float64
 4   Adj Close     4193 non-null   float64
 5   Volume        4193 non-null   int64  
 6   10_SMA        4184 non-null   float64
 7   10_WMA        4184 non-null   float64
 8   10_EMA        4193 non-null   float64
 9   Stochastic_K  4192 non-null   float64
 10  Stochastic_D  4192 non-null   float64
 11  RSI           4192 non-null   float64
dtypes: float64(11), int64(1)
memor

In [95]:
# macd-
# Define the window sizes for MACD calculation
short_window = 12  # 12-period for the short-term EMA
long_window = 26   # 26-period for the long-term EMA
signal_window = 9  # 9-period for the signal line

# Function to calculate MACD
def calculate_macd(data, short_window, long_window, signal_window):
    # Calculate the short-term EMA
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()

    # Calculate the long-term EMA
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()

    # Calculate the MACD line (difference between short-term EMA and long-term EMA)
    macd = short_ema - long_ema

    # Calculate the Signal line (9-period EMA of MACD)
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()

    # Calculate the MACD Histogram (MACD - Signal line)
    macd_histogram = macd - signal_line

    return macd, signal_line, macd_histogram

# Assuming your DataFrame is named 'data' and contains 'Close' column
data['MACD'], data['Signal_Line'], data['MACD_Histogram'] = calculate_macd(data, short_window, long_window, signal_window)

# Display the last few rows to check the calculations
print(data[[  'Close', 'MACD', 'Signal_Line', 'MACD_Histogram']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())


                 Close       MACD  Signal_Line  MACD_Histogram
Date                                                          
2022-07-18  490.299988   6.710410     5.010991        1.699419
2022-07-19  497.950012   7.657448     5.540282        2.117166
2022-07-20  508.600006   9.161738     6.264573        2.897164
2022-07-21  512.049988  10.511117     7.113882        3.397235
2022-07-22  513.700012  11.580164     8.007138        3.573025
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Open            4193 non-null   float64
 1   High            4193 non-null   float64
 2   Low             4193 non-null   float64
 3   Close           4193 non-null   float64
 4   Adj Close       4193 non-null   float64
 5   Volume          4193 non-null   int64  
 6   10_SMA          4184 non-null   float64
 7   10_WMA          4184 non

In [96]:
# larry williams--

# Define the window size for Williams %R calculation
williams_r_window = 14  # Typically, a 14-period window is used

# Function to calculate Williams %R using Close
def calculate_williams_r(data, window):
    # Calculate the highest price over the window
    high_n = data['Close'].rolling(window=window).max()
    
    # Calculate the lowest price over the window
    low_n = data['Close'].rolling(window=window).min()
    
    # Calculate Williams %R
    williams_r = ((high_n - data['Close']) / (high_n - low_n)) * -100
    
    return williams_r

# Assuming your DataFrame is named 'data' and contains 'Close' column
data['Williams_%R'] = calculate_williams_r(data, williams_r_window)

# Display the last few rows to check the calculations
print(data[[  'Close', 'Williams_%R']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())


                 Close  Williams_%R
Date                               
2022-07-18  490.299988         -0.0
2022-07-19  497.950012         -0.0
2022-07-20  508.600006         -0.0
2022-07-21  512.049988         -0.0
2022-07-22  513.700012         -0.0
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Open            4193 non-null   float64
 1   High            4193 non-null   float64
 2   Low             4193 non-null   float64
 3   Close           4193 non-null   float64
 4   Adj Close       4193 non-null   float64
 5   Volume          4193 non-null   int64  
 6   10_SMA          4184 non-null   float64
 7   10_WMA          4184 non-null   float64
 8   10_EMA          4193 non-null   float64
 9   Stochastic_K    4192 non-null   float64
 10  Stochastic_D    4192 non-null   float64
 11  RSI             4192 non-null   f

In [97]:
# A/d oscillator
print(data.columns)

def calculate_ad_oscillator(data):
    # Calculate the A/D oscillator value using the formula (HighPrice - ClosePrice(t-1)) / (HighPrice - LowPrice)
    data['A/D Oscillator'] = (data['High'] - data['Close'].shift(1)) / (data['High'] - data['Low'])
    return data


# Calculate the A/D Oscillator
data= calculate_ad_oscillator(data)

# Display the last few rows to check the calculations
print(data[[   'A/D Oscillator']].tail())

# Save the DataFrame to a CSV file
data.to_csv(f"{ticker}_final_data.csv", index=False)
print(data.info())

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '10_SMA',
       '10_WMA', '10_EMA', 'Stochastic_K', 'Stochastic_D', 'RSI', 'MACD',
       'Signal_Line', 'MACD_Histogram', 'Williams_%R'],
      dtype='object')
            A/D Oscillator
Date                      
2022-07-18        1.319149
2022-07-19        0.781991
2022-07-20        1.240194
2022-07-21        0.721805
2022-07-22        0.905558
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4193 entries, 2005-07-25 to 2022-07-22
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Open            4193 non-null   float64
 1   High            4193 non-null   float64
 2   Low             4193 non-null   float64
 3   Close           4193 non-null   float64
 4   Adj Close       4193 non-null   float64
 5   Volume          4193 non-null   int64  
 6   10_SMA          4184 non-null   float64
 7   10_WMA          4184 non-null   float64
 8   10_EMA 

In [98]:

print(data.columns)
print (data.shape)


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '10_SMA',
       '10_WMA', '10_EMA', 'Stochastic_K', 'Stochastic_D', 'RSI', 'MACD',
       'Signal_Line', 'MACD_Histogram', 'Williams_%R', 'A/D Oscillator'],
      dtype='object')
(4193, 17)


In [99]:
import pandas as pd


data = pd.read_csv(f'{ticker}_final_data.csv')
data=data[20:]
print(data.columns)
print(data.shape)
print(data.info())



predictors = data[['10_SMA',
       '10_WMA', '10_EMA', 'Stochastic_K', 'Stochastic_D', 'RSI', 'MACD',
       'Signal_Line', 'MACD_Histogram', 'Williams_%R', 'A/D Oscillator']].copy()
print(predictors.shape)


predictors.to_csv(f"{ticker}_predictors.csv")
predictors=pd.read_csv(f"{ticker}_predictors.csv")

response = data[['Close']].copy()

response['upordown'] = (response['Close'].diff().fillna(0) > 0).astype(float)

response.to_csv(f"{ticker}_response.csv", index=False)


response.to_csv(f"{ticker}_response.csv")
response=pd.read_csv(f"{ticker}_response.csv")
response=response["upordown"]
print(response)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '10_SMA',
       '10_WMA', '10_EMA', 'Stochastic_K', 'Stochastic_D', 'RSI', 'MACD',
       'Signal_Line', 'MACD_Histogram', 'Williams_%R', 'A/D Oscillator'],
      dtype='object')
(4173, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4173 entries, 20 to 4192
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Open            4173 non-null   float64
 1   High            4173 non-null   float64
 2   Low             4173 non-null   float64
 3   Close           4173 non-null   float64
 4   Adj Close       4173 non-null   float64
 5   Volume          4173 non-null   int64  
 6   10_SMA          4173 non-null   float64
 7   10_WMA          4173 non-null   float64
 8   10_EMA          4173 non-null   float64
 9   Stochastic_K    4173 non-null   float64
 10  Stochastic_D    4173 non-null   float64
 11  RSI             4173 non-null   float64
 12  MACD